In [3]:
import pandas as pd    
import geopandas as gpd
from geopandas.tools import geocode
#import geoplot
from shapely.geometry import Point
import numpy as np
import scipy.stats as stats
import scipy
import shapely
from shapely import speedups
speedups.enabled
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

**GeoJson files**

In [10]:
#Census tracts
census_url = 'https://opendata.arcgis.com/datasets/de58dc3e1efc49b782ab357e044ea20c_9.geojson'
census_bounds = gpd.read_file(census_url)

In [ ]:
#Bike lanes and walkways
#Bike lane index == 6
#Walkway index == 5

#df_SND = gpd.read_file('Street_Network_Database_SND.geojson')

Download at the link in the websites google doc and go to APIs to download as GeoJSON

In [11]:
#Bike racks
racks_url = 'https://opendata.arcgis.com/datasets/f86c29ce743e47819e588c3d643ceb63_0.geojson'
racks_data = gpd.read_file(racks_url)

In [ ]:
#Parking -- having some trouble with downloading this one, will explain in zoom

In [79]:
#Population -- don't have recent population data by Census tract, may have to estimate tract-specific
              #populations by relative area fraction of census tract from the total Seattle fraction
              #by year (this data we have).
        
seattle_pop = pd.read_csv('City_Annual_Stats.csv')

Population download link: https://data-seattlecitygis.opendata.arcgis.com/datasets/0920ecc5b00446629b55fd7e668d9b97_3.csv?outSR=%7B%22latestWkid%22%3A2926%2C%22wkid%22%3A2926%7D

_Traffic Flows_

In [15]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']

In [19]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = ['2015']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = ['2017']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT','geometry']]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [27]:
#Generate chronological stack of all traffic data

#I'm thinking for this we will do a for loop that iteratively stacks the aligned columns for every year

In [6]:
df_SND = gpd.read_file('Street_Network_Database_SND.geojson')

**Define census tract coordinate polygon areas:**

In [31]:
census_columns = ['NAME10', 'SHAPE_Area', 'geometry']
census_bounds_cleaned = census_bounds[census_columns]

In [32]:
census_bounds_cleaned.head()

,NAME10,SHAPE_Area,geometry
0,25,1.059462e+07,"POLYGON ((-122.29602 47.69023, -122.29608 47.6..."
1,26,1.339838e+07,"POLYGON ((-122.30817 47.69031, -122.30947 47.6..."
2,56,3.212601e+07,"POLYGON ((-122.39300 47.63956, -122.39421 47.6..."
3,68,7.729233e+06,"POLYGON ((-122.35070 47.63994, -122.35130 47.6..."
4,60,1.413816e+07,"POLYGON ((-122.34279 47.64320, -122.34280 47.6..."


**Once you're able to load in the relevant geoJson files:**

Send a message in the slack which one you wanna do. For right now I'm working on how to classify the locations from these files based on the census tract polygons. At this point we have to process:

1. Bike racks
3. Parking
4. Population

For each estimator we want year and geometric location. For example, if you run gdf_year(X) and head the file you will see the geometric coordinates. Refer to the code below for bike lanes specifically for a loose example of how we've approached this (or however you'd like, we are just shooting for consistent time series data of the estimators and their corresponding coordinates).

For cleaning based on year, we just have to make sure we are only using the 2007-2018 data so we will have to further filter the DFs to reflect that

_Bike lanes_

In [41]:
lane_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
bike = df_SND[lane_columns]

In [50]:
# Creates list of years in each row
lanes_years = []
for i in bike['SNDSEG_UPD'].values:
    lanes_years.append(int(i[0:4]))

In [51]:
bike['YEAR'] = pd.Series(lanes_years)

In [52]:
#Specify segment type to extract, for walkways this will be 5
bike = bike[bike['SEGMENT_TY']==6]

In [53]:
bike_cleaned = bike.drop(columns=['SNDSEG_UPD', 'SEGMENT_TY'])
bike_cleaned.head()

,SHAPE_Leng,geometry,YEAR
3,79.522621,"LINESTRING (-122.30780 47.61410, -122.30748 47...",2004
33,222.880092,"LINESTRING (-122.27182 47.66128, -122.27120 47...",2004
157,251.144111,"LINESTRING (-122.29444 47.59126, -122.29342 47...",2004
241,66.587768,"LINESTRING (-122.35275 47.62958, -122.35275 47...",2004
266,174.233452,"LINESTRING (-122.36442 47.62976, -122.36442 47...",2004


In [40]:
#for lon_lat in zip(bike_cleaned.geometry):
    #pt = Point(lon_lat)
    #print(lon_lat)
    
#Brendan -- currently working on converting input feature location to census tract class

_Walkways_

In [74]:
walk_columns = ['SEGMENT_TY', 'SNDSEG_UPD', 'SHAPE_Leng', 'geometry']
walk = df_SND[walk_columns]

In [75]:
walk_years = []
for i in racks['SNDSEG_UPD'].values:
    walk_years.append(int(i[0:4]))

In [76]:
walks['YEAR'] = pd.Series(walk_years)

In [77]:
walks = walks[walks['SEGMENT_TY']==5]

In [78]:
walks_cleaned = walks.drop(columns=['SNDSEG_UPD', 'SEGMENT_TY'])
walks_cleaned.head()

,SHAPE_Leng,geometry,YEAR
801,165.967550,"LINESTRING (-122.31491 47.69172, -122.31490 47...",NaN
8433,412.348264,"LINESTRING (-122.32653 47.64449, -122.32653 47...",NaN
12025,174.869554,"LINESTRING (-122.30436 47.57405, -122.30438 47...",NaN
17992,465.983637,"LINESTRING (-122.38645 47.54191, -122.38646 47...",NaN
18578,205.825219,"LINESTRING (-122.39685 47.56481, -122.39687 47...",NaN


_Bike racks_

_Parking_

_Population_